## Imports

In [1]:
import psycopg2

## Open connection to db

In [2]:
con = psycopg2.connect(database="movies", 
                       user="postgres", 
                       password="admin", 
                       host="postgres")
cursor = con.cursor()
print("Database opened successfully")

Database opened successfully


## Answer the questions

### 1. How many movies are in data set?

In [3]:
query = """
SELECT COUNT(DISTINCT movieid) 
FROM movies;
"""

cursor.execute(query)
result = cursor.fetchone()

print(f'There are {result[0]} unique movie ids.')

There are 9742 unique movie ids.


In [4]:
query = """
SELECT COUNT(DISTINCT title) 
FROM movies;
"""

cursor.execute(query)
result = cursor.fetchone()

print(f'There are {result[0]} unique movie titles.')

There are 9737 unique movie titles.


In [5]:
query = """
SELECT title, COUNT(*)
FROM movies
GROUP BY title
HAVING COUNT(*) > 1;
"""

cursor.execute(query)
result = cursor.fetchall()

print("There are duplicates in the table:")
for row in result:
    print(f"\t'{row[0]}' appears {row[1]} times.") 

There are duplicates in the table:
	'Eros (2004)' appears 2 times.
	'Saturn 3 (1980)' appears 2 times.
	'Confessions of a Dangerous Mind (2002)' appears 2 times.
	'Emma (1996)' appears 2 times.
	'War of the Worlds (2005)' appears 2 times.


In [6]:
print('Differences for duplicates: ')
for title, _ in result:
    
    query = f"""
    SELECT *
    FROM movies
    where title='{title}';
    """
    
    cursor.execute(query)
    result = cursor.fetchall()
    
    title = result[0][1]
    ids = [movieid for movieid, *_ in result]
    genres = [genre.split('|') for *_, genre in result]
    print(f'- {title} with ids {ids} of genres {genres}.')

Differences for duplicates: 
- Eros (2004) with ids [32600, 147002] of genres [['Drama'], ['Drama', 'Romance']].
- Saturn 3 (1980) with ids [2851, 168358] of genres [['Adventure', 'Sci-Fi', 'Thriller'], ['Sci-Fi', 'Thriller']].
- Confessions of a Dangerous Mind (2002) with ids [6003, 144606] of genres [['Comedy', 'Crime', 'Drama', 'Thriller'], ['Comedy', 'Crime', 'Drama', 'Romance', 'Thriller']].
- Emma (1996) with ids [838, 26958] of genres [['Comedy', 'Drama', 'Romance'], ['Romance']].
- War of the Worlds (2005) with ids [34048, 64997] of genres [['Action', 'Adventure', 'Sci-Fi', 'Thriller'], ['Action', 'Sci-Fi']].


### 2. What is the most common genre of movie?

In [7]:
query = """
SELECT UNNEST(STRING_TO_ARRAY(genres, '|')) AS genre, COUNT(*) AS amount
FROM movies
GROUP BY genre
ORDER BY amount DESC;
"""

cursor.execute(query)
result = cursor.fetchone()

print(f'{result[0]} is the most common movie genre.')

Drama is the most common movie genre.


### 3. What are top 10 movies with highest rate?

In [8]:
MIN_AMOUNT_OF_REVIEWS = 50

query = f"""
SELECT title, AVG(rating)::numeric(10,2) AS score, COUNT(title) AS reviews
FROM movies AS m
JOIN ratings AS r
ON m.movieid = r.movieid
GROUP BY title
HAVING count(title) > {MIN_AMOUNT_OF_REVIEWS}
ORDER BY score DESC
LIMIT 10;
"""

cursor.execute(query)
result = cursor.fetchall()

print(f'Top 10 highest score movies at minimum {MIN_AMOUNT_OF_REVIEWS} reviews:')
for title, score, reviews in result:
    print(f'\t{title} with score {score} and {reviews} reviews')

Top 10 highest score movies at minimum 50 reviews:
	Shawshank Redemption, The (1994) with score 4.43 and 317 reviews
	Godfather, The (1972) with score 4.29 and 192 reviews
	Fight Club (1999) with score 4.27 and 218 reviews
	Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) with score 4.27 and 97 reviews
	Cool Hand Luke (1967) with score 4.27 and 57 reviews
	Rear Window (1954) with score 4.26 and 84 reviews
	Godfather: Part II, The (1974) with score 4.26 and 129 reviews
	Departed, The (2006) with score 4.25 and 107 reviews
	Goodfellas (1990) with score 4.25 and 126 reviews
	Casablanca (1942) with score 4.24 and 100 reviews


### 4. What are 5 most often rating users?

In [9]:
query = """
SELECT userid, COUNT(*)
FROM ratings
GROUP BY userid
ORDER BY COUNT(*) DESC
LIMIT 5;
"""

cursor.execute(query)
result = cursor.fetchall()

print("Top 5 most often rating users:")
for place, (userid, qty) in enumerate(result):
    print(f'\t{place+1}. user {userid} with {qty} reviews')

Top 5 most often rating users:
	1. user 414 with 2698 reviews
	2. user 599 with 2478 reviews
	3. user 474 with 2108 reviews
	4. user 448 with 1864 reviews
	5. user 274 with 1346 reviews


### 5. When was done first and last rate included in data set and what was the rated movie tittle?

In [10]:
query = """
SELECT to_timestamp(timestamp), title
FROM ratings AS r
LEFT JOIN movies AS m
ON r.movieid = m.movieid
WHERE timestamp IN (SELECT min(timestamp) FROM ratings)
ORDER BY title;
"""

cursor.execute(query)
result = cursor.fetchall()

print(f"The first reviews were uploaded on {result[0][0]} UTC for:")
for row in result:
    print(f'\t{row[1]}')

The first reviews were uploaded on 1996-03-29 18:36:55+00:00 UTC for:
	Aladdin (1992)
	Apollo 13 (1995)
	Baby-Sitters Club, The (1995)
	Batman (1989)
	Beauty and the Beast (1991)
	Beverly Hills Cop III (1994)
	Black Beauty (1994)
	Boys on the Side (1995)
	Circle of Friends (1995)
	City Slickers II: The Legend of Curly's Gold (1994)
	Clear and Present Danger (1994)
	Cliffhanger (1993)
	Copycat (1995)
	Corrina, Corrina (1994)
	Crimson Tide (1995)
	Dances with Wolves (1990)
	Die Hard: With a Vengeance (1995)
	Disclosure (1994)
	Drop Zone (1994)
	Englishman Who Went Up a Hill But Came Down a Mountain, The (1995)


In [11]:
query = """
SELECT to_timestamp(timestamp), title
FROM ratings AS r
LEFT JOIN movies AS m
ON r.movieid = m.movieid
WHERE timestamp IN (SELECT max(timestamp) FROM ratings)
ORDER BY title;
"""

cursor.execute(query)
result = cursor.fetchall()

print(f"The last review was uploaded on {result[0][0]} UTC for:")
for row in result:
    print(f'\t{row[1]}')

The last review was uploaded on 2018-09-24 14:27:30+00:00 UTC for:
	Crumb (1994)


### 6. Find all movies released in 1990

In [12]:
query = """
SELECT DISTINCT title
FROM movies
WHERE title LIKE '%(1990)';
"""

cursor.execute(query)
result = cursor.fetchall()

print("List of movies released in 1990:")
for title in result:
    print(f'\t{title[0]}')

List of movies released in 1990:
	Adventures of Ford Fairlane, The (1990)
	Air America (1990)
	Akira Kurosawa's Dreams (Dreams) (1990)
	Alice (1990)
	All the Vermeers in New York (1990)
	Almost an Angel (1990)
	Amityville Curse, The (1990)
	Angel at My Table, An (1990)
	Another 48 Hrs. (1990)
	Arachnophobia (1990)
	Avalon (1990)
	Awakenings (1990)
	Back to the Future Part III (1990)
	Begotten (1990)
	Betsy's Wedding (1990)
	Bird on a Wire (1990)
	Blue Steel (1990)
	Bonfire of the Vanities (1990)
	Book of Love (1990)
	Bride of Re-Animator (1990)
	Bullet in the Head (1990)
	Cadillac Man (1990)
	Captain America (1990)
	Child's Play 2 (1990)
	Civil War, The (1990)
	Come See the Paradise (1990)
	Cry-Baby (1990)
	Cyrano de Bergerac (1990)
	Dances with Wolves (1990)
	Darkman (1990)
	Days of Being Wild (A Fei jingjyuhn) (1990)
	Days of Thunder (1990)
	Delta Force 2 (Delta Force 2: The Colombian Connection) (1990)
	Dick Tracy (1990)
	Die Hard 2 (1990)
	Dragon Ball Z: Bardock - The Father of Gok

## Close connection to database

In [13]:
con.close()